# Modelado de Clasificación - Predicción de Estrés

En este notebook, se construirán y evaluarán varios modelos de clasificación para predecir el nivel de estrés.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings

warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

try:
    train_df = pd.read_csv('datos/train.csv')
    test_df = pd.read_csv('datos/test.csv')
    
    print("Archivos 'train.csv' y 'test.csv' cargados exitosamente.")
    print(f"Muestras de entrenamiento: {len(train_df)}")
    print(f"Muestras de prueba: {len(test_df)}")
    
except FileNotFoundError:
    print("Error: No se encontraron los archivos 'train.csv' o 'test.csv'.")
    print("Por favor, asegúrate de que estén en la carpeta 'datos/'.")

print("\n--- Columnas y tipos de datos (Train) ---")
train_df.info()

print("\n--- Primeras filas (Train) ---")
display(train_df.head())

## 1. Ingeniería de Características y Preprocesamiento

In [ ]:
print("\n--- Paso 1: Ingeniería y Preprocesamiento ---")

def engineer_bp(df):
    bp_split = df['Blood Pressure'].str.split('/', expand=True).astype(float)
    df['Systolic_BP'] = bp_split[0]
    df['Diastolic_BP'] = bp_split[1]
    df = df.drop('Blood Pressure', axis=1)
    return df

print("Aplicando ingeniería a 'Blood Pressure'...")
X_train_featured = engineer_bp(train_df.drop('Stress_Category', axis=1))
X_test_featured = engineer_bp(test_df.drop('Stress_Category', axis=1))

target_map = {'ESTRESADO': 1, 'ESTRES_MODERADO': 0}
y_train = train_df['Stress_Category'].map(target_map)
y_test = test_df['Stress_Category'].map(target_map)

print("Variable objetivo mapeada a 1 (Estresado) y 0 (Moderado).")

numeric_features = ['Age', 'Sleep Duration', 'Quality of Sleep', 
                    'Physical Activity Level', 'Heart Rate', 'Daily Steps',
                    'Systolic_BP', 'Diastolic_BP']

categorical_features = ['Gender', 'Occupation', 'BMI Category', 'Sleep Disorder']

ALL_FEATURES = numeric_features + categorical_features

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

print("Pipeline de preprocesamiento creado.")

## 2. Entrenamiento del Modelo Base (Regresión Logística)

In [ ]:
print("\n--- Paso 2: Entrenar Modelo Base (Logistic Regression) ---")

lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

print("Entrenando el modelo...")
lr_pipeline.fit(X_train_featured, y_train)

print("Generando predicciones sobre los datos de TRAIN...")
y_pred_train_lr = lr_pipeline.predict(X_train_featured)

print("Modelo base entrenado y predicciones (de train) generadas.")

## 3. Métricas del Modelo Base

In [ ]:
print("\n--- Paso 3: Métricas del Modelo Base (sobre datos de TRAIN) ---")

cm_lr = confusion_matrix(y_train, y_pred_train_lr)
print(f"Matriz de Confusión (Train):\n{cm_lr}\n")

sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Moderado (0)', 'Estresado (1)'], 
            yticklabels=['Moderado (0)', 'Estresado (1)'])
plt.ylabel('Valor Real')
plt.xlabel('Predicción')
plt.title('Matriz de Confusión - Modelo Base (Train)')
plt.show()

tn, fp, fn, tp = cm_lr.ravel()
print(f"Verdaderos Negativos (TN): {tn} (Predijo 'Moderado' y era 'Moderado')")
print(f"Falsos Positivos   (FP): {fp} (Predijo 'Estresado' y era 'Moderado')")
print(f"Falsos Negativos   (FN): {fn} (Predijo 'Moderado' y era 'Estresado')")
print(f"Verdaderos Positivos (TP): {tp} (Predijo 'Estresado' y era 'Estresado')\n")

precision = precision_score(y_train, y_pred_train_lr, zero_division=0)
recall = recall_score(y_train, y_pred_train_lr, zero_division=0)
f1 = f1_score(y_train, y_pred_train_lr, zero_division=0)
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

print(f"--- Métricas Calculadas (Train) ---")
print(f"Precision:   {precision:.4f}")
print(f"Recall (Sensibilidad): {recall:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"F1-Score:    {f1:.4f}\n")

print("--- Reporte de Clasificación (Train) ---")
print(classification_report(y_train, y_pred_train_lr, target_names=['Moderado (0)', 'Estresado (1)'], zero_division=0))

## 4. Entrenamiento y Comparación de Modelos

In [ ]:
print("\n--- Paso 4: Entrenar y Comparar 4 Modelos (evaluados en TEST) ---")

models = {
    "Regresión Logística": LogisticRegression(random_state=42, max_iter=1000),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "Support Vector Machine (SVM)": SVC(random_state=42, probability=True)
}

results_list = []

print("Entrenando y evaluando modelos en el set de PRUEBA...")

for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    pipeline.fit(X_train_featured, y_train)
    
    y_pred_test = pipeline.predict(X_test_featured)
    
    accuracy = accuracy_score(y_test, y_pred_test)
    precision = precision_score(y_test, y_pred_test, zero_division=0)
    recall = recall_score(y_test, y_pred_test, zero_division=0)
    f1 = f1_score(y_test, y_pred_test, zero_division=0)
    
    cm = confusion_matrix(y_test, y_pred_test)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    results_list.append({
        "Modelo": name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall (Sen)": recall,
        "F1-Score": f1,
        "Specificity": specificity
    })

results_df = pd.DataFrame(results_list).set_index("Modelo")

print("\n--- Tabla Comparativa de Métricas (TEST) ---")
display(results_df)

## 5. Análisis de Error

In [ ]:
print("\n--- Paso 6: Análisis de Error (usando Gradient Boosting) ---")

best_model_name = "Gradient Boosting"
best_model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(random_state=42))
])

best_model_pipeline.fit(X_train_featured, y_train)
y_pred_best = best_model_pipeline.predict(X_test_featured)

error_df = X_test_featured.copy()
error_df['Actual_Stress'] = y_test
error_df['Predicted_Stress'] = y_pred_best

map_inv = {1: 'ESTRESADO', 0: 'ESTRES_MODERADO'}
error_df['Actual_Stress'] = error_df['Actual_Stress'].map(map_inv)
error_df['Predicted_Stress'] = error_df['Predicted_Stress'].map(map_inv)

fn_errors = error_df[
    (error_df['Actual_Stress'] == 'ESTRESADO') & 
    (error_df['Predicted_Stress'] == 'ESTRES_MODERADO')
]

print(f"El modelo ({best_model_name}) tuvo {len(fn_errors)} Falsos Negativos (los peores errores) en el set de prueba.")
print("Mostrando estos errores:")
display(fn_errors)

print("\n--- Cómo analizar estos errores ---")
print("Observa la Fila 0 (o cualquier fila de arriba):")
print("1. La persona estaba 'ESTRESADA' (Actual), pero el modelo dijo 'MODERADO' (Predicted).")
print("2. Mira sus características: 'Quality of Sleep' = 6, 'Sleep Duration' = 6.2, 'Sleep Disorder' = 'None'.")
print("3. Explicación posible: El modelo aprendió que tener 'Sleep Disorder' = 'None' es un fuerte indicador de estrés 'Moderado'.")
print("   Aunque su calidad y duración de sueño eran bajas (6 y 6.2, típicos de estrés), el 'None' en el trastorno pudo")
print("   confundir al modelo y darle más peso, llevando a la predicción incorrecta.")

## 6. Guardar el Modelo

In [ ]:
print("\n--- Paso 7: Guardar Modelo y Crear UI ---")

model_filename = 'stress_model_pipeline.pkl'
joblib.dump(best_model_pipeline, model_filename)

print(f"Pipeline del modelo '{best_model_name}' guardado como '{model_filename}'")